<a href="https://colab.research.google.com/github/AIkikr/code-translation-pipeline-2/blob/main/%E8%A8%80%E8%AA%9E%E3%82%BF%E3%82%B9%E3%82%AF_%EF%BC%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# python 3.10.12
!pip install -U pip
!pip install -U transformers
!pip install -U bitsandbytes
!pip install -U accelerate
!pip install -U datasets
!pip install -U peft
!pip install -U trl
!pip install -U wandb
!pip install ipywidgets --upgrade

In [2]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    get_peft_model,
)
import os, torch, gc
from datasets import load_dataset
import bitsandbytes as bnb
from trl import SFTTrainer

from transformers import DataCollatorForLanguageModeling


In [3]:
from google.colab import drive, files
from getpass import getpass
import os
from google.colab import userdata

hf_token = userdata.get('HF_TOKEN')

# トークンを環境変数に設定
os.environ['HUGGINGFACE_HUB_TOKEN'] = hf_token

# CLIにログイン
!huggingface-cli login --token $HUGGINGFACE_HUB_TOKEN

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `HF_TOKEN` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `HF_TOKEN`


In [4]:
from IPython.display import Javascript
#JavaScriptコードを埋め込む
keep_alive_code = """
function keepColabAlive() {
    setInterval(() => {
        document.querySelector("colab-toolbar-button#connect").click();
    }, 600000); // 10分ごとにクリック
}
keepColabAlive();
"""

# JavaScriptを実行
display(Javascript(keep_alive_code))

<IPython.core.display.Javascript object>

In [5]:
# モデルを読み込み。
# llm-jp-3 1.8B, 3.7B, 13Bのsnapshotをダウンロード済みでmodelsディレクトリに格納してあります。
# base_model_idの値はomnicampusの環境におけるモデルのパスを表しており、それ以外の環境で実行する場合は変更の必要があります。
# その他のモデルは取得に承諾が必要なため、各自でダウンロードお願いします。
base_model_id = "microsoft/phi-4" #Fine-Tuningするベースモデル
# omnicampus以外の環境をご利用の方は以下をご利用ください。
# base_model_id = "llm-jp/llm-jp-3-13b"
new_model_id = "phi-4-4bit-Q-lora-kura-lang-3" #Fine-Tuningしたモデルにつけたい名前

In [6]:
from google.colab import drive
drive.mount('/content/drive')

# チェックポイント保存用のディレクトリを作成
!mkdir -p /content/drive/MyDrive/code_translation_project/checkpoints

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
# Google Driveからデータセットをダウンロード
!pip install gdown
!gdown 1xw6Edqf_nknKoei_LC49n4EtvNQezKGe

# ダウンロードしたファイルを解凍
!unzip -q codetransocean_datasets.zip -d ./data
!ls -la

Downloading...
From (original): https://drive.google.com/uc?id=1xw6Edqf_nknKoei_LC49n4EtvNQezKGe
From (redirected): https://drive.google.com/uc?id=1xw6Edqf_nknKoei_LC49n4EtvNQezKGe&confirm=t&uuid=f5f466d6-5449-4472-9ddc-d94df42b8c9d
To: /content/CodeTransOcean Datasets.zip
100% 132M/132M [00:03<00:00, 34.2MB/s]
unzip:  cannot find or open codetransocean_datasets.zip, codetransocean_datasets.zip.zip or codetransocean_datasets.zip.ZIP.
total 128872
drwxr-xr-x 1 root root      4096 Mar 22 10:16  .
drwxr-xr-x 1 root root      4096 Mar 22 09:40  ..
-rw-r--r-- 1 root root 131942209 Oct 24  2023 'CodeTransOcean Datasets.zip'
drwxr-xr-x 4 root root      4096 Mar 20 13:31  .config
drwx------ 6 root root      4096 Mar 22 10:07  drive
drwxr-xr-x 1 root root      4096 Mar 20 13:31  sample_data


In [6]:
# ダウンロードしたZIPファイルを解凍
!unzip -q "CodeTransOcean Datasets.zip"

# 解凍後のディレクトリとファイルを確認
!ls -la

replace __MACOSX/._CodeTrans Datasets? [y]es, [n]o, [A]ll, [N]one, [r]ename: total 128880
drwxr-xr-x 1 root root      4096 Mar 22 10:16  .
drwxr-xr-x 1 root root      4096 Mar 22 09:40  ..
drwxr-xr-x 6 root root      4096 Oct 23  2023 'CodeTrans Datasets'
-rw-r--r-- 1 root root 131942209 Oct 24  2023 'CodeTransOcean Datasets.zip'
drwxr-xr-x 4 root root      4096 Mar 20 13:31  .config
drwx------ 6 root root      4096 Mar 22 10:07  drive
drwxr-xr-x 3 root root      4096 Mar 22 10:16  __MACOSX
drwxr-xr-x 1 root root      4096 Mar 20 13:31  sample_data


In [7]:
# データセットディレクトリの内容を確認
!ls -la "CodeTrans Datasets"

total 36
drwxr-xr-x 6 root root 4096 Oct 23  2023 .
drwxr-xr-x 1 root root 4096 Mar 22 10:16 ..
drwxr-xr-x 2 root root 4096 Oct 23  2023 DLTrans
-rw-rw-r-- 1 root root 8196 Oct 23  2023 .DS_Store
drwxrwxr-x 2 root root 4096 Aug 28  2023 LLMTrans
drwxr-xr-x 2 root root 4096 Oct 23  2023 MultilingualTrans
drwxr-xr-x 2 root root 4096 Oct 23  2023 NicheTrans


In [8]:
# MultilingualTransディレクトリの内容を確認
!ls -la "CodeTrans Datasets/MultilingualTrans"

total 151184
drwxr-xr-x 2 root root     4096 Oct 23  2023 .
drwxr-xr-x 6 root root     4096 Oct 23  2023 ..
-rw-rw-r-- 1 root root 44466928 May 28  2023 multilingual_test.json
-rw-rw-r-- 1 root root 91702782 May 28  2023 multilingual_train.json
-rw-rw-r-- 1 root root 18623972 May 28  2023 multilingual_valid.json


In [9]:
# ファイルの最初の数行を読み込み、構造を確認
import json

# トレーニングファイルの構造を確認
train_file = "CodeTrans Datasets/MultilingualTrans/multilingual_train.json"
with open(train_file, 'r', encoding='utf-8') as file:
    first_line = file.readline().strip()
    item = json.loads(first_line)
    print("ファイル内の最初のエントリのキー:", list(item.keys()))
    print("言語キーの例：", [key for key in item.keys() if key not in ["id", "name"]])

    # 言語サンプルを表示（最初のいくつかの文字）
    for lang_key in [key for key in item.keys() if key not in ["id", "name"]]:
        print(f"\n{lang_key} サンプル (先頭100文字):")
        print(item[lang_key][:100])

ファイル内の最初のエントリのキー: ['id', 'name', 'C', 'C#']
言語キーの例： ['C', 'C#']

C サンプル (先頭100文字):
#include<graphics.h>
#include<stdlib.h>
#include<math.h>

typedef struct{
	double x,y;
}complex;

co

C# サンプル (先頭100文字):
using System.Drawing;



using System.Linq;

namespace RosettaJuliaSet
{
    class Program
    {
   


In [10]:
# ファイル全体のすべての言語キーを収集
def collect_all_language_keys(file_path):
    all_keys = set()
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                item = json.loads(line.strip())
                # id や name のような非言語キーを除外
                keys = [key for key in item.keys() if key not in ["id", "name"]]
                all_keys.update(keys)
            except json.JSONDecodeError:
                continue
    return sorted(list(all_keys))

# すべての言語キーを収集
all_language_keys = collect_all_language_keys(train_file)
print("データセット内のすべての言語キー:", all_language_keys)

# すべての可能な言語ペアを作成
all_pairs = []
for i, source in enumerate(all_language_keys):
    for target in all_language_keys:
        if source != target:
            all_pairs.append((source, target))

print("\n生成した言語ペア (最初の10ペア):", all_pairs[:10])
print(f"合計言語ペア数: {len(all_pairs)}")

データセット内のすべての言語キー: ['C', 'C#', 'C++', 'Go', 'Java', 'PHP', 'Python', 'VB']

生成した言語ペア (最初の10ペア): [('C', 'C#'), ('C', 'C++'), ('C', 'Go'), ('C', 'Java'), ('C', 'PHP'), ('C', 'Python'), ('C', 'VB'), ('C#', 'C'), ('C#', 'C++'), ('C#', 'Go')]
合計言語ペア数: 56


In [11]:
# JSONLファイルからデータセットを読み込む関数
def load_jsonl_dataset(file_path, lang_pairs):
    """JSONLファイルからデータセットを読み込む関数"""
    import json
    import pandas as pd
    from datasets import Dataset, DatasetDict

    all_data = {f"{source}_to_{target}": [] for source, target in lang_pairs}

    # ファイルを行ごとに読み込む
    line_count = 0
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                item = json.loads(line.strip())
                line_count += 1

                # 各言語ペアに対してデータを収集
                for source, target in lang_pairs:
                    if source in item and target in item:
                        all_data[f"{source}_to_{target}"].append({
                            "source_code": item[source],
                            "target_code": item[target],
                            "source_lang": source,
                            "target_lang": target
                        })
            except json.JSONDecodeError:
                print(f"JSONデコードエラー: {line[:50]}...")
                continue

    print(f"処理した行数: {line_count}")

    # DatasetDict形式に変換
    result = {}
    for pair_name, data_list in all_data.items():
        if data_list:
            result[pair_name] = Dataset.from_pandas(pd.DataFrame(data_list))
            print(f"言語ペア {pair_name} のデータを読み込みました: {len(data_list)} サンプル")

    return DatasetDict(result)

# 選択した言語ペアでデータセットを読み込む
selected_pairs = [
    ("C++", "Python"),
    ("Python", "C++"),
    ("Java", "Python"),
    ("Python", "Java"),
    ("C#", "Python"),
    ("Python", "C#")
]

# トレーニングとテストのファイルパスが正しいことを確認
train_file = "CodeTrans Datasets/MultilingualTrans/multilingual_train.json"
test_file = "CodeTrans Datasets/MultilingualTrans/multilingual_test.json"

print("\n選択した言語ペア:", selected_pairs)
train_dataset = load_jsonl_dataset(train_file, selected_pairs)
test_dataset = load_jsonl_dataset(test_file, selected_pairs)


選択した言語ペア: [('C++', 'Python'), ('Python', 'C++'), ('Java', 'Python'), ('Python', 'Java'), ('C#', 'Python'), ('Python', 'C#')]
処理した行数: 38230
言語ペア C++_to_Python のデータを読み込みました: 1684 サンプル
言語ペア Python_to_C++ のデータを読み込みました: 1684 サンプル
言語ペア Java_to_Python のデータを読み込みました: 1676 サンプル
言語ペア Python_to_Java のデータを読み込みました: 1676 サンプル
言語ペア C#_to_Python のデータを読み込みました: 1626 サンプル
言語ペア Python_to_C# のデータを読み込みました: 1626 サンプル
処理した行数: 15090
言語ペア C++_to_Python のデータを読み込みました: 810 サンプル
言語ペア Python_to_C++ のデータを読み込みました: 810 サンプル
言語ペア Java_to_Python のデータを読み込みました: 872 サンプル
言語ペア Python_to_Java のデータを読み込みました: 872 サンプル
言語ペア C#_to_Python のデータを読み込みました: 398 サンプル
言語ペア Python_to_C# のデータを読み込みました: 398 サンプル


In [12]:
# 以下のコードは「print("\n選択した言語ペア:", selected_pairs)」の後に挿入

# 必要なライブラリを確実にインポート
!pip install -q datasets pandas tqdm

# インポート文を正しく記載
from datasets import load_dataset, Dataset, DatasetDict, concatenate_datasets
import pandas as pd
from tqdm import tqdm

# 追加データセットのための新しい言語ペアを定義
# CodeXGLUEデータセットではJavaとC#のみが利用可能なため、それらの言語ペアを追加
additional_pairs = [
    ("Java", "C#"),
    ("C#", "Java")
]

# 元の言語ペアと追加言語ペアを結合
extended_pairs = selected_pairs + additional_pairs
print("拡張言語ペア:", extended_pairs)

# CodeXGLUE TransCoderデータセットをHugging Faceから取得
print("CodeXGLUEのデータセットをロード中...")

# CodeXGLUEのTransCoderデータセットを読み込む
try:
    codexglue_dataset = load_dataset("google/code_x_glue_cc_code_to_code_trans")
    print(f"CodeXGLUEデータセットをロードしました: {codexglue_dataset}")
except Exception as e:
    print(f"CodeXGLUEデータセット読み込みエラー: {e}")
    codexglue_dataset = None

# CodeXGLUEのデータを変換する関数
def convert_codexglue_format(dataset):
    """CodeXGLUEデータセットをCodeTransOcean形式に変換"""
    if dataset is None:
        return None

    # 言語マッピング（CodeXGLUEの言語名をCodeTransOceanの形式に合わせる）
    lang_mapping = {
        "java": "Java",
        "cs": "C#"
    }

    # 変換結果を格納する辞書
    converted_pairs = {}

    # データセットの構造を確認
    first_item = next(iter(dataset['train']))
    available_langs = [key for key in first_item.keys() if key != 'id']
    print(f"利用可能な言語: {available_langs}")

    # 各スプリット（train, validation, test）を処理
    for split_name in dataset:
        split_data = dataset[split_name]
        print(f"処理中のスプリット: {split_name}, サンプル数: {len(split_data)}")

        # このスプリットの変換データを格納
        converted_split = {}

        # 言語ペア用のデータを初期化
        for source_lang in available_langs:
            s_lang = lang_mapping[source_lang]
            for target_lang in available_langs:
                if source_lang != target_lang:
                    t_lang = lang_mapping[target_lang]
                    pair_key = f"{s_lang}_to_{t_lang}"
                    converted_split[pair_key] = []

        # 各サンプルから言語ペアを生成
        for item in tqdm(split_data, desc=f"変換中 {split_name}"):
            for source_lang in available_langs:
                s_lang = lang_mapping[source_lang]
                for target_lang in available_langs:
                    if source_lang != target_lang:
                        t_lang = lang_mapping[target_lang]
                        pair_key = f"{s_lang}_to_{t_lang}"

                        converted_split[pair_key].append({
                            "source_code": item[source_lang],
                            "target_code": item[target_lang],
                            "source_lang": s_lang,
                            "target_lang": t_lang
                        })

        # データセットに変換
        pair_datasets = {}
        for pair_key, items in converted_split.items():
            if items:
                pair_datasets[pair_key] = Dataset.from_pandas(pd.DataFrame(items))
                print(f"{split_name} データ {pair_key}: {len(items)} サンプル")

        # 変換結果に追加
        converted_pairs[split_name] = pair_datasets

    return converted_pairs

# CodeXGLUEデータセットを変換
if codexglue_dataset:
    print("CodeXGLUEデータセットを変換中...")
    codexglue_converted = convert_codexglue_format(codexglue_dataset)
else:
    codexglue_converted = None

# 元のCodeTransOceanデータセットを読み込む
print("CodeTransOceanデータセットを読み込み中...")
train_dataset = load_jsonl_dataset(train_file, extended_pairs)
test_dataset = load_jsonl_dataset(test_file, extended_pairs)

# 元のCodeTransOceanデータセットから検証データを作成する関数
def create_validation_split(dataset_dict, validation_ratio=0.1, random_seed=42):
    """
    各言語ペアのデータセットから一定割合を検証用データとして分割する

    Parameters:
    -----------
    dataset_dict : DatasetDict
        元のデータセット辞書
    validation_ratio : float
        検証用に分割する割合 (0-1)
    random_seed : int
        乱数シード

    Returns:
    --------
    train_dict : DatasetDict
        分割後のトレーニングデータセット辞書
    validation_dict : DatasetDict
        分割後の検証用データセット辞書
    """
    train_dict = {}
    validation_dict = {}

    for pair_key, dataset in dataset_dict.items():
        # データセットをシャッフルしてからトレーニングと検証に分割
        shuffled_dataset = dataset.shuffle(seed=random_seed)
        split_datasets = shuffled_dataset.train_test_split(
            test_size=validation_ratio,
            seed=random_seed
        )

        # 分割結果をそれぞれの辞書に格納
        train_dict[pair_key] = split_datasets['train']
        validation_dict[f"{pair_key}_validation"] = split_datasets['test']

        print(f"{pair_key}: {len(dataset)} → トレーニング {len(split_datasets['train'])}, 検証 {len(split_datasets['test'])}")

    return train_dict, validation_dict

# CodeTransOceanデータセットから検証データを分割
print("\nCodeTransOceanデータセットから検証データを作成...")
train_datasets, validation_datasets = create_validation_split(
    train_dataset,
    validation_ratio=0.1,  # 10%を検証用に分割
    random_seed=42
)

# データセットを統合する
if codexglue_converted and "train" in codexglue_converted:
    print("\nCodeXGLUEデータセットとの統合を開始します...")

    # CodeXGLUEデータセットのトレーニングデータを統合
    for pair_key, dataset in codexglue_converted["train"].items():
        if pair_key in train_datasets:
            # 既存の言語ペアにデータを追加
            original_size = len(train_datasets[pair_key])
            train_datasets[pair_key] = concatenate_datasets([
                train_datasets[pair_key],
                dataset
            ])
            new_size = len(train_datasets[pair_key])
            print(f"トレーニング {pair_key}: {original_size} → {new_size} サンプルに増加 (+{new_size - original_size})")
        else:
            # 新しい言語ペアを追加
            train_datasets[pair_key] = dataset
            print(f"新しいトレーニングペア {pair_key}: {len(dataset)} サンプル追加")

    # CodeXGLUEデータセットの検証データを統合
    if "validation" in codexglue_converted:
        for pair_key, dataset in codexglue_converted["validation"].items():
            validation_key = f"{pair_key}_validation"
            if validation_key in validation_datasets:
                # 既存の検証データに追加
                original_size = len(validation_datasets[validation_key])
                validation_datasets[validation_key] = concatenate_datasets([
                    validation_datasets[validation_key],
                    dataset
                ])
                new_size = len(validation_datasets[validation_key])
                print(f"検証 {validation_key}: {original_size} → {new_size} サンプルに増加 (+{new_size - original_size})")
            else:
                # 新しい検証データを追加
                validation_datasets[validation_key] = dataset
                print(f"新しい検証ペア {validation_key}: {len(dataset)} サンプル追加")

# 全てのデータセットを最終的な辞書に統合
final_datasets = {}

# トレーニングデータを追加
for pair_key, dataset in train_datasets.items():
    final_datasets[pair_key] = dataset

# 検証データを追加
for validation_key, dataset in validation_datasets.items():
    final_datasets[validation_key] = dataset

# テストデータを追加
for pair_key, dataset in test_dataset.items():
    test_key = f"{pair_key}_test"
    final_datasets[test_key] = dataset
    print(f"テストデータ {test_key}: {len(dataset)} サンプル")

# データ量の多い言語ペアをサンプリング
print("\nデータ量の多い言語ペアをサンプリングします...")
# サンプリング対象の言語ペア
sampling_pairs = ["Java_to_C#", "C#_to_Java"]
max_samples = 5000

# データセットをサンプリング
for pair_key in sampling_pairs:
    if pair_key in final_datasets and len(final_datasets[pair_key]) > max_samples:
        original_size = len(final_datasets[pair_key])
        # ランダムにサンプリング
        sampled_dataset = final_datasets[pair_key].shuffle(seed=42).select(range(max_samples))
        final_datasets[pair_key] = sampled_dataset
        print(f"  {pair_key}: {original_size} → {max_samples} サンプルに削減 (-{original_size - max_samples})")

# 最終的なデータセットサイズを表示
print("\n統合後の最終データセット:")
total_samples = {"train": 0, "validation": 0, "test": 0}

for key, dataset in final_datasets.items():
    samples = len(dataset)
    if "_validation" in key:
        dataset_type = "validation"
    elif "_test" in key:
        dataset_type = "test"
    else:
        dataset_type = "train"

    total_samples[dataset_type] += samples
    print(f"  {key}: {samples} サンプル")

print(f"\n合計: トレーニング {total_samples['train']}, 検証 {total_samples['validation']}, テスト {total_samples['test']} サンプル")

# train_datasetを新しいデータセットで置き換え
train_dataset = DatasetDict(final_datasets)

拡張言語ペア: [('C++', 'Python'), ('Python', 'C++'), ('Java', 'Python'), ('Python', 'Java'), ('C#', 'Python'), ('Python', 'C#'), ('Java', 'C#'), ('C#', 'Java')]
CodeXGLUEのデータセットをロード中...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


CodeXGLUEデータセットをロードしました: DatasetDict({
    train: Dataset({
        features: ['id', 'java', 'cs'],
        num_rows: 10300
    })
    validation: Dataset({
        features: ['id', 'java', 'cs'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'java', 'cs'],
        num_rows: 1000
    })
})
CodeXGLUEデータセットを変換中...
利用可能な言語: ['java', 'cs']
処理中のスプリット: train, サンプル数: 10300


変換中 train: 100%|██████████| 10300/10300 [00:00<00:00, 29277.88it/s]


train データ Java_to_C#: 10300 サンプル
train データ C#_to_Java: 10300 サンプル
処理中のスプリット: validation, サンプル数: 500


変換中 validation: 100%|██████████| 500/500 [00:00<00:00, 25631.60it/s]


validation データ Java_to_C#: 500 サンプル
validation データ C#_to_Java: 500 サンプル
処理中のスプリット: test, サンプル数: 1000


変換中 test: 100%|██████████| 1000/1000 [00:00<00:00, 29773.23it/s]


test データ Java_to_C#: 1000 サンプル
test データ C#_to_Java: 1000 サンプル
CodeTransOceanデータセットを読み込み中...
処理した行数: 38230
言語ペア C++_to_Python のデータを読み込みました: 1684 サンプル
言語ペア Python_to_C++ のデータを読み込みました: 1684 サンプル
言語ペア Java_to_Python のデータを読み込みました: 1676 サンプル
言語ペア Python_to_Java のデータを読み込みました: 1676 サンプル
言語ペア C#_to_Python のデータを読み込みました: 1626 サンプル
言語ペア Python_to_C# のデータを読み込みました: 1626 サンプル
言語ペア Java_to_C# のデータを読み込みました: 1500 サンプル
言語ペア C#_to_Java のデータを読み込みました: 1500 サンプル
処理した行数: 15090
言語ペア C++_to_Python のデータを読み込みました: 810 サンプル
言語ペア Python_to_C++ のデータを読み込みました: 810 サンプル
言語ペア Java_to_Python のデータを読み込みました: 872 サンプル
言語ペア Python_to_Java のデータを読み込みました: 872 サンプル
言語ペア C#_to_Python のデータを読み込みました: 398 サンプル
言語ペア Python_to_C# のデータを読み込みました: 398 サンプル
言語ペア Java_to_C# のデータを読み込みました: 348 サンプル
言語ペア C#_to_Java のデータを読み込みました: 348 サンプル

CodeTransOceanデータセットから検証データを作成...
C++_to_Python: 1684 → トレーニング 1515, 検証 169
Python_to_C++: 1684 → トレーニング 1515, 検証 169
Java_to_Python: 1676 → トレーニング 1508, 検証 168
Python_to_Java: 1676 → トレーニング 1508, 検証 168
C#_to_P

In [15]:
# 量子化設定
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# モデルとトークナイザーを読み込む
model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [16]:
# LoRA適用モジュールを特定する関数
def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit
    lora_module_names = set()

    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    if 'lm_head' in lora_module_names:
        lora_module_names.remove('lm_head')

    return list(lora_module_names)

modules = find_all_linear_names(model)

In [17]:
# LoRA設定
peft_config = LoraConfig(
    r=4,
    lora_alpha=8,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=modules
)

# LoRAをモデルに適用
model = get_peft_model(model, peft_config)

# プロンプトの定義
prompt = """### Instruction
Translate from {source_lang} to {target_lang}:
{source_code}

### Translation
{target_code}"""

EOS_TOKEN = tokenizer.eos_token


In [18]:
def format_data(examples):
    inputs = [
        prompt.format(
            source_lang=source_lang,
            target_lang=target_lang,
            source_code=source_code,
            target_code=target_code
        ) + EOS_TOKEN
        for source_code, target_code, source_lang, target_lang in zip(
            examples["source_code"],
            examples["target_code"],
            examples["source_lang"],
            examples["target_lang"]
        )
    ]
    return tokenizer(inputs, max_length=512, padding="max_length", truncation=True)

In [19]:
from datasets import concatenate_datasets

# 統合された単一のデータセットを作成
train_dataset_all = concatenate_datasets([ds for ds in train_dataset.values()])
test_dataset_all = concatenate_datasets([ds for ds in test_dataset.values()])

# 統合後に前処理
train_dataset_processed = train_dataset_all.map(
    format_data,
    batched=True,
    remove_columns=["source_code", "target_code", "source_lang", "target_lang"]
)

test_dataset_processed = test_dataset_all.map(
    format_data,
    batched=True,
    remove_columns=["source_code", "target_code", "source_lang", "target_lang"]
)


Map:   0%|          | 0/26128 [00:00<?, ? examples/s]

Map:   0%|          | 0/4856 [00:00<?, ? examples/s]

In [20]:
# 学習設定
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/code_translation_project/checkpoints",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    num_train_epochs=1,
    logging_steps=10,
    warmup_steps=20,
    save_steps=100,
    save_total_limit=2,
    learning_rate=2e-5,
    fp16=False,
    bf16=True,
    group_by_length=True,
    report_to="none"
)

In [21]:
# SFTTrainerを使用したFine-Tuning
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset_processed,
    eval_dataset=test_dataset_processed,
    peft_config=peft_config,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False
    )
)

Converting train dataset to ChatML:   0%|          | 0/26128 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/26128 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/26128 [00:00<?, ? examples/s]

Converting eval dataset to ChatML:   0%|          | 0/4856 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/4856 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/4856 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [22]:
import logging
logging.getLogger("transformers").setLevel(logging.ERROR)


In [ ]:
# キャッシュを無効にして学習を開始
model.config.use_cache = False
trainer.train()

Step,Training Loss
10,0.542400
20,0.583400
30,0.596200
40,0.581900
50,0.512900
60,0.500100
70,0.508900
80,0.577200
90,0.565800
100,0.493000


Step,Training Loss
10,0.542400
20,0.583400
30,0.596200
40,0.581900
50,0.512900
60,0.500100
70,0.508900
80,0.577200
90,0.565800
100,0.493000


In [8]:
# Google Driveのマウント
from google.colab import drive
import os
from google.colab import userdata

# ドライブをマウント
drive.mount('/content/drive', force_remount=True)


# Hugging Faceトークンの取得
try:
    hf_token = userdata.get('HF_TOKEN')
    os.environ['HUGGINGFACE_HUB_TOKEN'] = hf_token
except Exception as e:
    print(f"トークン取得エラー: {e}")
    # 手動入力のバックアップ方法
    from getpass import getpass
    hf_token = getpass("Hugging Faceトークンを入力してください: ")

# CLIにログイン
!huggingface-cli login --token $HUGGINGFACE_HUB_TOKEN

Mounted at /content/drive
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `HF_TOKEN` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `HF_TOKEN`


In [5]:


# 必要なライブラリのインポート
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel, LoraConfig, get_peft_model

# チェックポイントディレクトリのパス
checkpoint_dir = "/content/drive/MyDrive/code_translation_project/checkpoints/checkpoint-5600"



In [6]:
# ベースモデル名
base_model_id = "microsoft/phi-4"

# 4ビット量子化の設定
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# ベースモデルとトークナイザーの読み込み
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(base_model_id)



config.json:   0%|          | 0.00/802 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.4k [00:00<?, ?B/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00002-of-00006.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/1.61M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/917k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.25M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.50k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/95.0 [00:00<?, ?B/s]

以下、Colabを誤って途中で切断したため、確認及び暫定対処

In [11]:
import os

# チェックポイントディレクトリの内容を確認
checkpoint_dir = "/content/drive/MyDrive/code_translation_project/checkpoints/checkpoint-5600"
print("チェックポイントディレクトリの内容:")
print(os.listdir(checkpoint_dir))

# ファイルの詳細情報
for filename in os.listdir(checkpoint_dir):
    filepath = os.path.join(checkpoint_dir, filename)
    print(f"\nファイル: {filename}")
    print(f"サイズ: {os.path.getsize(filepath)} バイト")

チェックポイントディレクトリの内容:
['README.md', 'adapter_model.safetensors', 'adapter_config.json', 'tokenizer_config.json', 'special_tokens_map.json', 'vocab.json', 'merges.txt', 'tokenizer.json', 'training_args.bin', 'optimizer.pt', 'scheduler.pt', 'rng_state.pth', 'trainer_state.json']

ファイル: README.md
サイズ: 5089 バイト

ファイル: adapter_model.safetensors
サイズ: 55748320 バイト

ファイル: adapter_config.json
サイズ: 807 バイト

ファイル: tokenizer_config.json
サイズ: 17799 バイト

ファイル: special_tokens_map.json
サイズ: 463 バイト

ファイル: vocab.json
サイズ: 1612637 バイト

ファイル: merges.txt
サイズ: 916646 バイト

ファイル: tokenizer.json
サイズ: 7153083 バイト

ファイル: training_args.bin
サイズ: 5624 バイト

ファイル: optimizer.pt
サイズ: 111601914 バイト

ファイル: scheduler.pt
サイズ: 1064 バイト

ファイル: rng_state.pth
サイズ: 14244 バイト

ファイル: trainer_state.json
サイズ: 125399 バイト


In [13]:
from safetensors.torch import load_file
import json
from peft import LoraConfig, get_peft_model

# チェックポイントディレクトリのパス
checkpoint_dir = "/content/drive/MyDrive/code_translation_project/checkpoints/checkpoint-5600"

# アダプター設定ファイルのパス
adapter_config_path = os.path.join(checkpoint_dir, "adapter_config.json")
adapter_model_path = os.path.join(checkpoint_dir, "adapter_model.safetensors")

# アダプター設定の読み込み
with open(adapter_config_path, 'r') as f:
    adapter_config_dict = json.load(f)

# LoRA設定の再現
lora_config = LoraConfig(**adapter_config_dict)

# LoRAモデルの準備
model = get_peft_model(base_model, lora_config)

# SafeTensorsでの重みの読み込み
try:
    state_dict = load_file(adapter_model_path, device='cuda')

    # モデルの現在のアダプターを削除
    if hasattr(model, 'peft_config'):
        model.unload()

    # 重みを読み込み
    model.load_state_dict(state_dict, strict=False)
    print("アダプター重みを読み込みました。")

except Exception as e:
    print(f"重みの読み込み中にエラーが発生: {e}")

アダプター重みを読み込みました。


In [15]:
# Hugging Faceトークンの取得
from google.colab import userdata
hf_token = userdata.get('HF_TOKEN')

# Hugging Faceにアップロードするモデル名
new_model_id = "AkiK/phi-4-4bit-Q-lora-kura-lang-3"  # バージョン番号を更新

# モデルとトークナイザーをHugging Faceにアップロード
try:
    model.push_to_hub(new_model_id, token=hf_token, private=True)
    tokenizer.push_to_hub(new_model_id, token=hf_token, private=True)
    print(f"モデルを {new_model_id} としてHugging Faceにアップロードしました。")
except Exception as e:
    print(f"アップロード中にエラーが発生: {e}")

adapter_model.safetensors:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


モデルを AkiK/phi-4-4bit-Q-lora-kura-lang-3 としてHugging Faceにアップロードしました。


In [10]:
# モジュール名を自動検出する関数
def find_all_linear_names(model):
    import bitsandbytes as bnb
    cls = bnb.nn.Linear4bit
    lora_module_names = set()

    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    if 'lm_head' in lora_module_names:
        lora_module_names.remove('lm_head')

    return list(lora_module_names)

# LoRAモジュールの特定
modules = find_all_linear_names(base_model)

# LoRA設定
peft_config = LoraConfig(
    r=4,
    lora_alpha=8,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=modules
)

# LoRAモデルの準備
model = get_peft_model(base_model, peft_config)

# チェックポイントから重みを読み込み
adapter_path = os.path.join(checkpoint_dir, "adapter_model.safetensors")
adapter_config_path = os.path.join(checkpoint_dir, "adapter_config.json")

# 重みの読み込み
if os.path.exists(adapter_path):
    try:
        # weights_only=Falseを明示的に指定
        state_dict = torch.load(adapter_path, weights_only=False, map_location='cuda')

        # モデルの現在のアダプターを削除
        if hasattr(model, 'peft_config'):
            model.unload()

        # 重みを読み込み
        model.load_state_dict(state_dict, strict=False)
        print("アダプター重みを読み込みました。")
    except Exception as e:
        print(f"重みの読み込み中にエラーが発生: {e}")
else:
    print("アダプター重みファイルが見つかりません。")

# Hugging Faceにアップロードするモデル名
new_model_id = "AkiK/phi-4-4bit-Q-lora-kura-lang-3"

# モデルとトークナイザーをHugging Faceにアップロード
try:
    model.push_to_hub(new_model_id, token=hf_token, private=True)
    tokenizer.push_to_hub(new_model_id, token=hf_token, private=True)
    print(f"モデルを {new_model_id} としてHugging Faceにアップロードしました。")
except Exception as e:
    print(f"アップロード中にエラーが発生: {e}")

重みの読み込み中にエラーが発生: invalid load key, '\xd8'.


README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

モデルを AkiK/phi-4-4bit-Q-lora-kura-lang-3 としてHugging Faceにアップロードしました。


In [31]:
from safetensors.torch import load_file
import json
from peft import LoraConfig, get_peft_model

# チェックポイントディレクトリのパス
checkpoint_dir = "/content/drive/MyDrive/code_translation_project/checkpoints/checkpoint-5600"

# アダプター設定ファイルのパス
adapter_config_path = os.path.join(checkpoint_dir, "adapter_config.json")
adapter_model_path = os.path.join(checkpoint_dir, "adapter_model.safetensors")

# アダプター設定の読み込み
with open(adapter_config_path, 'r') as f:
    adapter_config_dict = json.load(f)

# LoRAモデルの準備（最初のアダプターを完全に削除）
if hasattr(model, 'peft_config'):
    model = model.unload()  # 既存のアダプターを完全に削除

# LoRA設定の再現
lora_config = LoraConfig(**adapter_config_dict)

# 新しいLoRAアダプターを追加
model = get_peft_model(base_model, lora_config)

# SafeTensorsでの重みの読み込み
try:
    state_dict = load_file(adapter_model_path, device='cuda')

    # 重みを読み込み（strict=Falseに注意）
    model.load_state_dict(state_dict, strict=False)
    print("アダプター重みを読み込みました。")

except Exception as e:
    print(f"重みの読み込み中にエラーが発生: {e}")

アダプター重みを読み込みました。


In [30]:
pip install safetensors datasets

In [38]:
!pip install sacrebleu

In [41]:
import os
import torch
import gc
import json
import random
import warnings
import pandas as pd
import sacrebleu
from datasets import Dataset, DatasetDict
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel, LoraConfig, get_peft_model
from safetensors.torch import load_file

# トークナイザーの読み込み時
tokenizer = AutoTokenizer.from_pretrained(base_model_id, padding_side="left")
tokenizer.pad_token = tokenizer.eos_token

# メモリ解放関数
def free_memory():
    torch.cuda.empty_cache()
    gc.collect()

# BLEU計算関数
def calculate_bleu(predictions, references):
    references_list = [[ref] for ref in references]
    bleu = sacrebleu.corpus_bleu(predictions, references_list)
    return {"bleu": bleu.score}

# 生成と評価関数
def generate_and_evaluate(model, tokenizer, test_data, num_samples=50, batch_size=4):
    random.seed(42)
    test_data = random.sample(test_data, min(num_samples, len(test_data)))

    predictions = []
    references = []

    for i in range(0, len(test_data), batch_size):
        batch_data = test_data[i:i+batch_size]
        batch_prompts = [item["prompt"] for item in batch_data]

        # トークン化時にパディングを左側に
        inputs = tokenizer(
            batch_prompts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=512,
            padding_side="left"  # 明示的に左側パディングを指定
        ).to(model.device)

        with torch.no_grad():
            try:
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=512,
                    temperature=0.2,
                    do_sample=True,
                    top_p=0.95,
                    repetition_penalty=1.1,
                    num_return_sequences=1
                )
            except RuntimeError as e:
                print(f"生成中にエラー: {e}")
                free_memory()
                continue

        for j, output in enumerate(outputs):
            try:
                generated_text = tokenizer.decode(output, skip_special_tokens=True)
                prompt_length = len(batch_data[j]["prompt"])
                predicted_code = generated_text[prompt_length:].strip()

                code_markers = ["```python", "```cpp", "```java", "```cs", "```c#", "```"]
                for marker in code_markers:
                    if marker in predicted_code:
                        predicted_code = predicted_code.split(marker)[1].split("```")[0].strip()
                        break

                predictions.append(predicted_code)
                references.append(batch_data[j]["target"])
            except Exception as e:
                print(f"コード抽出中にエラー: {e}")

        free_memory()

    bleu_score = calculate_bleu(predictions, references)
    return bleu_score, predictions, references

# JSONLファイルからデータセットを読み込む関数
def load_jsonl_dataset(file_path, lang_pairs):
    all_data = {f"{source}_to_{target}": [] for source, target in lang_pairs}

    line_count = 0
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                item = json.loads(line.strip())
                line_count += 1

                for source, target in lang_pairs:
                    if source in item and target in item:
                        all_data[f"{source}_to_{target}"].append({
                            "source_code": item[source],
                            "target_code": item[target],
                            "source_lang": source,
                            "target_lang": target
                        })
            except json.JSONDecodeError:
                print(f"JSONデコードエラー: {line[:50]}...")
                continue

    print(f"処理した行数: {line_count}")

    result = {}
    for pair_name, data_list in all_data.items():
        if data_list:
            result[pair_name] = Dataset.from_pandas(pd.DataFrame(data_list))
            print(f"言語ペア {pair_name} のデータを読み込みました: {len(data_list)} サンプル")

    return DatasetDict(result)

# 選択した言語ペア
selected_pairs = [
    ("C++", "Python"),
    ("Python", "C++"),
    ("Java", "Python"),
    ("Python", "Java"),
    ("C#", "Python"),
    ("Python", "C#")
]

# トレーニングとテストのファイルパスを設定
train_file = "CodeTrans Datasets/MultilingualTrans/multilingual_train.json"
test_file = "CodeTrans Datasets/MultilingualTrans/multilingual_test.json"

print("\n選択した言語ペア:", selected_pairs)
train_dataset = load_jsonl_dataset(train_file, selected_pairs)
test_dataset = load_jsonl_dataset(test_file, selected_pairs)

test_dataset_with_prompt = []

# 最終的なデータセット（final_datasets）からテストデータを取得
for key, dataset in test_dataset.items():
    for item in dataset:
        prompt = f"""### Instruction
Translate from {item['source_lang']} to {item['target_lang']}:
{item['source_code']}

### Translation"""

        test_dataset_with_prompt.append({
            "prompt": prompt,
            "target": item['target_code']
        })

# テストデータのサンプル数を印刷
print(f"テストデータのサンプル数: {len(test_dataset_with_prompt)}")

# 評価処理
from datetime import datetime
import os
import pandas as pd

# 評価処理
try:
    bleu_score, predictions, references = generate_and_evaluate(
        model,
        tokenizer,
        test_dataset_with_prompt,
        num_samples=len(test_dataset_with_prompt),  # 全サンプルを使用
        batch_size=4
    )

    # 現在の日時を取得
    current_time = datetime.now().strftime("%Y%m%d_%H%M%S")

    # 保存先ディレクトリを作成
    save_dir = "/content/drive/MyDrive/code_translation_project/evaluation_results_phi_4_SFT"
    os.makedirs(save_dir, exist_ok=True)

    # BLEUスコアをテキストファイルに保存
    bleu_file_path = os.path.join(save_dir, f"bleu_score_{current_time}.txt")
    with open(bleu_file_path, 'w') as f:
        f.write(f"BLEUスコア: {bleu_score['bleu']}")

    # 予測結果をCSVに保存
    results_df = pd.DataFrame({
        'Prompt': [item['prompt'] for item in test_dataset_with_prompt],
        'Prediction': predictions,
        'Reference': references,
        'BLEU_Score': [bleu_score['bleu']] * len(predictions)
    })

    csv_file_path = os.path.join(save_dir, f"evaluation_results_{current_time}.csv")
    results_df.to_csv(csv_file_path, index=False, encoding='utf-8')

    # コンソールに結果を表示
    print(f"BLEUスコア: {bleu_score['bleu']}")
    print(f"評価結果を {csv_file_path} に保存しました。")
    print(f"BLEUスコアを {bleu_file_path} に保存しました。")

    # 最初の5サンプルを表示
    for i in range(min(5, len(predictions))):
        print(f"\n--- サンプル {i+1} ---")
        print("元のコード:", test_dataset_with_prompt[i]["prompt"])
        print("\n予測されたコード:\n", predictions[i])
        print("\n正解のコード:\n", references[i])

except Exception as e:
    print(f"評価中に致命的なエラーが発生: {e}")
finally:
    free_memory()


選択した言語ペア: [('C++', 'Python'), ('Python', 'C++'), ('Java', 'Python'), ('Python', 'Java'), ('C#', 'Python'), ('Python', 'C#')]
処理した行数: 38230
言語ペア C++_to_Python のデータを読み込みました: 1684 サンプル
言語ペア Python_to_C++ のデータを読み込みました: 1684 サンプル
言語ペア Java_to_Python のデータを読み込みました: 1676 サンプル
言語ペア Python_to_Java のデータを読み込みました: 1676 サンプル
言語ペア C#_to_Python のデータを読み込みました: 1626 サンプル
言語ペア Python_to_C# のデータを読み込みました: 1626 サンプル
処理した行数: 15090
言語ペア C++_to_Python のデータを読み込みました: 810 サンプル
言語ペア Python_to_C++ のデータを読み込みました: 810 サンプル
言語ペア Java_to_Python のデータを読み込みました: 872 サンプル
言語ペア Python_to_Java のデータを読み込みました: 872 サンプル
言語ペア C#_to_Python のデータを読み込みました: 398 サンプル
言語ペア Python_to_C# のデータを読み込みました: 398 サンプル
テストデータのサンプル数: 4160


KeyboardInterrupt: 

以上、Colabを切断したための暫定対処

上記不測自体がなかった場合、下記コードでアップロード/評価予定だった

In [ ]:

# 出力ディレクトリをGoogleドライブに変更
training_args.output_dir = "/content/drive/MyDrive/phi_code_translator"

In [ ]:
# モデルの保存
trainer.save_model("/content/drive/MyDrive/phi_code_translator/final_model")

In [ ]:
pip install --upgrade evaluate

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [13]:
import torch
import gc
import random
import warnings
import sacrebleu
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
from google.colab import output

# メモリ解放と最適化の関数
def free_memory():
    torch.cuda.empty_cache()
    gc.collect()

# 警告の非表示と初期メモリ解放
warnings.filterwarnings('ignore', category=UserWarning)
free_memory()

# Colabのウィジェットマネージャーを有効化
output.enable_custom_widget_manager()

# ベースモデル名
base_model_id = "microsoft/phi-4"

# 4ビット量子化の設定（メモリ効率化）
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True  # さらなるメモリ削減
)

# デバイスマップの設定
device_map = {
    "": 0,  # すべてのレイヤーをGPU 0に
    "lm_head": 0
}

# モデルとトークナイザーの読み込み（メモリ効率化）
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    device_map=device_map,
    low_cpu_mem_usage=True
)

tokenizer = AutoTokenizer.from_pretrained(base_model_id, padding_side="left")

# トークナイザーの追加設定
tokenizer.pad_token = tokenizer.eos_token

# LoRAアダプターのリポジトリ名
lora_model_name = "AkiK/phi-4-4bit-Q-lora-kura-lang-3"

# PeftModelとしてLoRAアダプターを読み込み
try:
    model = PeftModel.from_pretrained(
        base_model,
        lora_model_name,
        is_trainable=False,
        device_map=device_map
    )
except Exception as e:
    print(f"アダプター読み込みエラー: {e}")
    model = base_model

# モデルの最終準備
model.eval()  # 評価モードに
model = model.to('cuda')
free_memory()

# BLEU計算用の関数
def calculate_bleu(predictions, references):
    references_list = [[ref] for ref in references]
    bleu = sacrebleu.corpus_bleu(predictions, references_list)
    return {"bleu": bleu.score}

# 予測と評価の関数（バッチ処理とメモリ効率化）
def generate_and_evaluate(model, tokenizer, test_data, num_samples=50, batch_size=4):
    # ランダムサンプリング
    random.seed(42)
    test_data = random.sample(test_data, min(num_samples, len(test_data)))

    predictions = []
    references = []

    # バッチ処理
    for i in range(0, len(test_data), batch_size):
        batch_data = test_data[i:i+batch_size]
        batch_prompts = [item["prompt"] for item in batch_data]

        # バッチでのトークン化
        inputs = tokenizer(
            batch_prompts,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=512
        ).to(model.device)

        # バッチでの生成
        with torch.no_grad():
            try:
                outputs = model.generate(
                    **inputs,
                    max_new_tokens=512,  # トークン数削減
                    temperature=0.2,
                    do_sample=True,
                    top_p=0.95,
                    repetition_penalty=1.1,
                    num_return_sequences=1
                )
            except RuntimeError as e:
                print(f"生成中にエラー: {e}")
                free_memory()
                continue

        # バッチ処理の結果を処理
        for j, output in enumerate(outputs):
            try:
                generated_text = tokenizer.decode(output, skip_special_tokens=True)
                prompt_length = len(batch_data[j]["prompt"])
                predicted_code = generated_text[prompt_length:].strip()

                # コードブロック抽出
                code_markers = ["```python", "```cpp", "```java", "```cs", "```c#", "```"]
                for marker in code_markers:
                    if marker in predicted_code:
                        predicted_code = predicted_code.split(marker)[1].split("```")[0].strip()
                        break

                predictions.append(predicted_code)
                references.append(batch_data[j]["target"])
            except Exception as e:
                print(f"コード抽出中にエラー: {e}")

        # 各バッチ後にメモリを解放
        free_memory()

    # BLEUスコアの計算
    bleu_score = calculate_bleu(predictions, references)

    return bleu_score, predictions, references

# テストデータの準備
# テストデータの準備
test_dataset_with_prompt = []

# final_datasetsからテストデータを抽出
test_keys = [key for key in final_datasets.keys() if "_test" in key]

for key in test_keys:
    dataset = final_datasets[key]
    for item in dataset:
        # プロンプトの形式を合わせる
        prompt = f"""### Instruction
Translate from {item['source_lang']} to {item['target_lang']}:
{item['source_code']}

### Translation"""

        test_dataset_with_prompt.append({
            "prompt": prompt,
            "target": item['target_code']
        })

print(f"評価用データ準備完了: {len(test_dataset_with_prompt)} サンプル")

# メイン評価処理
try:
    # テストデータでの評価
    bleu_score, predictions, references = generate_and_evaluate(
        model,
        tokenizer,
        test_dataset_with_prompt,
        num_samples=50,
        batch_size=4
    )
    print(f"BLEUスコア: {bleu_score}")

    # 予測結果の詳細表示
    for i in range(min(5, len(predictions))):
        print(f"\n--- サンプル {i+1} ---")
        print("元のコード:", test_dataset_with_prompt[i]["prompt"])
        print("\n予測されたコード:\n", predictions[i])
        print("\n正解のコード:\n", references[i])

except Exception as e:
    print(f"評価中に致命的なエラーが発生: {e}")
finally:
    free_memory()

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/807 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

評価用データ準備完了: 4856 サンプル
BLEUスコア: {'bleu': 45.61810157218275}

--- サンプル 1 ---
元のコード: ### Instruction
Translate from C++ to Python:
#include <iostream>
#include <fstream>
#include <cmath>

using namespace std;

string readFile (string path) {
    string contents;
    string line;
    ifstream inFile(path);
    while (getline (inFile, line)) {
        contents.append(line);
        contents.append("\n");
    }
    inFile.close();
    return contents;
}

double entropy (string X) {
    const int MAXCHAR = 127;
    int N = X.length();
    int count[MAXCHAR];
    double count_i;
    char ch;
    double sum = 0.0;
    for (int i = 0; i < MAXCHAR; i++) count[i] = 0;
    for (int pos = 0; pos < N; pos++) {
        ch = X[pos];
        count[(int)ch]++;
    }
    for (int n_i = 0; n_i < MAXCHAR; n_i++) {
        count_i = count[n_i];
        if (count_i > 0) sum -= count_i / N * log2(count_i / N);
    }
    return sum;
}

int main () {
    cout<<entropy(readFile("entropy.cpp"));
    return 0;
}




トークン数をおさえたバージョン

In [ ]:
import json

results = {
    'bleu_score': bleu_score['bleu'],
    'predictions': predictions,
    'references': references
}

with open('/content/drive/MyDrive/code_translation_results_2.json', 'w') as f:
    json.dump(results, f, indent=4)
print("結果をJSONファイルに保存しました。")

結果をJSONファイルに保存しました。
